# 8.1 더 깊게
## 8.1.1 더 깊은 신경망으로
아래 그림과 같이 구성된 CNN을 만들고자 한다.

<img src="https://velog.velcdn.com/post-images%2Fdscwinterstudy%2Fbb9c3a30-4192-11ea-82e0-f10b8886fb3a%2F%EC%86%90%EA%B8%80%EC%94%A8%EC%8B%AC%EC%B8%B5CNN.png" width=800 height=400></img><br><br> 

여기에서 사용하는 합성곱 계층은 모두 3x3 크기의 작은 필터로, 층이 깊어지면서 채널 수가 더 늘어나는 것이 특징이다(합성곱 계층의 채널 수는 앞 계층에서부터 순서대로 16, 16, 32, 32, 64, 64로 늘어간다). 또 그림과 같이 풀링 계층을 추가하여 중간 데이터의 공간 크기를 점차 줄여간다. 긜고 마지막 단의 완전연결 계층에서는 드롭아웃 계층을 사용한다.

가중치 초깃값으로 He 초깃값을 사용하고, 가중치 매개변수 갱신에는 Adam을 사용한다.

- 3x3의 작은 필터를 사용한 합성곱 계층
- 활성화 함수는 ReLU
- 완전연결 계층 뒤에 드롭아웃 계층 사용
- Adam을 사용해 최적화
- 가중치 초기값은 'He의 초기값'

이 신경망의 정확도는 99.38% 가 된다.<BR>
이 신경망을 구현한 소스 코드는 deep_convet.py에, 또 훈련용 코드는 train_deepnet.py에 있다.

## 8.1.2 정확도를 더 높이려면
앙상블 학습, 학습률 감소, 데이터 확장 등이 정확도 향상에 공헌한다.

**데이터 확장(data augmentation)** 은 입력 이미지(훈련 이미지)를 알고리즘을 동원해 '인위적'으로 확장한다. 입력 이미지를 회전하거나 세로로 이동하는 등 미세한 변화를 주어 이미지 개수를 늘린다. 데이터가 몇 개 없을 때 효과적인 수단이다.

- 회전에 의한 변형(rotate)

- 이동에 의한 변형(translation)

- 이미지 일부를 잘라낸다(crop)

- 좌우를 뒤집는다(flip) - 이미지의 대칭성을 고려하지 않아도 되는 경우에만 쓸 수 있다.
- 크기수정(rescale)

- 밝기변화(lighting condition) 등

## 8.1.3 깊게 하는 이유
층을 깊게 할 때의 이점을 알아보자

신경망의 매개변수 수가 줄어든다. 층을 깊게 한 신경망은 깊지 않은 경우보다 적은 매개변수로 같은(혹은 그 이상) 수준의 표현력을 달성할 수 있다. 합성곱 연산에서의 필터 크기에 주목해 생각해보면 쉽게 이해할 것이다. 예를 들어보자

<img src="https://velog.velcdn.com/post-images%2Fdscwinterstudy%2Fd4af9990-4192-11ea-bf7e-239e36e1bc11%2F5by5%ED%95%A9%EC%84%B1%EA%B3%B1.png" width=800 height=400></img><br><br> 

위 그림은 5x5 필터로 구성된 합성곱 계층이다.<br><Br>
여기에서 주목한 점은 출력 데이터의 각 노드가 입력 데이터의 어느 영역으로부터 계산되었느냐는 것이다. 당연하지만 위 그림의 예에서는 각각의 출력 노드는 입력 데이터의 5x5 크기 영역에서 계산된다.

<img src="https://velog.velcdn.com/post-images%2Fdscwinterstudy%2Fd9594b80-4192-11ea-b7ab-b932b78555b8%2F3by3%ED%95%A9%EC%84%B1%EA%B3%B1.png" width=800 height=250></img><br><br> 

위 그림은 3x3의 합성곱 연산을 2회 반복하는 경우이다. 이 경우 출력 노드 하나는 중간 데이터의 3x3 영역에서 계산된다. 그럼 중간 데이터의 3x3 영역은 그 전 입력 데이터의 어느 영역에서 계산될까? 5x5 크기의 영역에서 계산되어 나온다. 즉, 출력 데이터는 입력 데이터의 5x5 영역을 '보고' 계산하게 된다.

5x5의 합성곱 연산 1회는 3x3의 합성곱 연산 2회 수행하여 대체할 수 있다. 게다가 전자의 매개변수 수가 25개(5x5)인 반면, 후자는 총 18개(2x3x3)이며, 매개변수 수는 층을 반복할수록 적어진다. 그리고 그 개수의 차이는 층이 깊어질수록 커진다. 예를 들어 3x3의 합성곱 연산을 3회 반복하면 매개변수는 모두 27개가 되지만, 같은 크기의 영역을 1회의 합성곱 연산으로 '보기' 위해서는 7x7 크기의 필터, 즉 배개변수 49개가 필요하다.

**NOTE_** 작은 필터를 겹쳐 신경망을 깊게 할 때의 장점은 매개변수 수를 줄여 넓은 **수용 영역(receptive field)** 을 소화할 수 있다는 데 있다(수용 영역은 뉴런에 변화를 일으키는 국소적인 공간 영역이다). 게다가 층을 거듭하면서 ReLU 등의 활성화 함수를 합성곱 계층 사이에 끼움으로써 신경망의 표현력이 개선된다. 이는 활성화 함수가 신경망에 '비선형' 힘을 가하고, 비선형 함수가 겹치면서 더 복잡한 것도 표현할 수 있기 때문이다.

신경망을 깊게 하면 학습해야 할 문제를 계층적으로 분해할 수 있다. 각 층이 학습해야 할 문제를 더 단순한 문제로 대체할 수 있는 것이다. 

층을 깊게 하면 정보를 계층적으로 전달할 수 있다. 예를 들어 에지를 추출한 층의 다음 층은 에지 정보를 쓸 수 있고, 더 고도의 패턴을 효과적으로 학습하리라 기대할 수 있다. 즉, 층을 깊이 함으로써 각 층이 학습해야 할 문제를 '풀기 쉬운 단순한 문제'로 분해할 수 있어 효율적으로 학습하리라 기대할 수 있다.

# 8.2 대표적인 신경망
## 8.2.1 VGG
성곱 계층과 풀링 계층으로 구성되는 기본적인 CNN이다.
비중 있는 층(합성곱 계층, 완전연결 계층)을 모두 16층(혹은 19층)으로 심화한 게 특징이다.


<img src="https://velog.velcdn.com/post-images%2Fdscwinterstudy%2Fea967bb0-4193-11ea-82e0-f10b8886fb3a%2FVGG.png" width=800 height=300></img><br><br> 

3x3의 작은 필터를 사용한 합성곱 계층을 연속으로 거친다.
합성곱 계층을 2~4회 연속으로 풀링 계층을 두어 크기를 절반으로 줄이는 처리를 반복하고 마지막에는 완전연결 계층을 통과시켜 결과를 출력한다.

## 8.2.2 GoogLeNet
기본적으로는 기존의 CNN과 다르지 않지만 세로 방향 깊이 뿐 아니라 가로 방향도 깊다는 점이 특징이다.<br>
GoogLeNet에는 가로 방향에 폭이 있고 이를 인셉셥 구조라 한다.


<img src="https://velog.velcdn.com/post-images%2Fdscwinterstudy%2F8556b550-419b-11ea-a3b8-fd5b14e3378a%2F%EA%B5%AC%EA%B8%80%EB%84%B7%EC%9D%B8%EC%85%89%EC%85%98.png" width=800 height=300></img><br><br> 

인셉션 구조는 위와 같이 크기가 다른 필터(와 풀링)를 여러 개 적용하여 그 결과를 결합한다. 이 인셉션 구조를 하나의 빌딩 블록(구성요소)으로 사용하는 것이 GoogLeNet의 특징이다.<BR><BR>
또한, GoogLeNet에서는 1x1 크기의 필터를 사용한 합성곱 계층을 많은 곳에서 사용한다. 이 1x1의 합성곱 연산은 채널 쪽으로 크기를 줄이는 것으로, 매개변수 제거와 고속 처리에 기여한다.

## 8.2.3 ResNet
<img src="https://velog.velcdn.com/post-images%2Fdscwinterstudy%2Ffb4e1da0-4193-11ea-9037-c1df5ac05aa2%2FResNet.png" width=500 height=300></img><br><br> 

층을 깊게 하는 것이 성능 향상에 도움이 되지만 지나치게 깊게 하면 오히려 성능이 떨어진다. 이런 문제를 해결하기 위해 **스킵 연결(skip connection)** 을 도입한다.

**스킵연결(skip connection):** 층의 깊이에 비례해 성능을 향상시킨다.<BR>
입력 데이터를 합성곱 계층(weight layer)을 건너뛰어 출력에 바로 더하는 구조<BR>
역전파 때 스킵 연결이 신호 감쇠를 막아주기 때문에 층이 깊어져도 학습을 효율적으로 할 수 있다.

**NOTE_** 스킵 연결은 입력 데이터를 '그대로' 흘리는 것으로, 역전파 때도 상류의 기울기를 그대로 하류로 보낸다. 여기에서의 핵심은 상류의 기울기에 아무런 수정도 가하지 않고 '그대로' 흘린다는 것이다. 그래서 스킵 연결로 기울기가 작아지거나 지나치게 커질 걱정 없이 앞 층에 '의미 있는 기울기'가 전해질 수 있다. 층을 깊게 할수록 기울기가 작아지는 소실 문제를 줄여줄 수 있다.

**NOTE_** 이미지넷이 제공하는 거대한 데이터셋으로 학습한 가중치 값들은 실제 제품에 활용해도 효과적이다. 이를 **전이 학습(transfer learning)** 이라고 해서, 학습된 가중치(혹은 그 일부)를 다른 신경망에 복사한 다음, 그 상대로 재학습을 수행한다. 예를 들어 VGG와 구성이 같은 신경망을 준비하고, 미리 학습된 가중치를 초깃값으로 설정한 후, 새로운 데이터셋을 대상으로 재학습(fine tuning)을 수행한다. 전이 학습은 보유한 데이터셋이 적을 때 특히 유용한 방법이다.